In [1]:
import pandas as pd
import numpy as np
import re

from collections import defaultdict

In [2]:
# Read KS data collected in January
# ks_df = pd.read_csv('./kickstarter_data/01.csv', index_col=0)
# relevant_categories = ['design/product design', 'fashion/apparel', 'fashion/accessories', 'fashion', 'technology/gadgets', 'fashion/footwear', 'fashion/ready-to-wear', 'art/textiles', 'fashion/childrenswear', 'fashion/couture', 'technology/wearables']
# relevant_df = ks_df[ks_df.category.isin(relevant_categories)]

In [3]:
# Read Amzn Data
amz_df = pd.read_csv('./amazon_data/Duffel.csv')
amz_df_tt = pd.read_csv('./amazon_data/TravelTote.csv')
amz_df_luggage = pd.read_csv('./amazon_data/Luggage.csv')

In [4]:
new_df = pd.concat([amz_df, amz_df_tt, amz_df_luggage], ignore_index=True)

In [5]:
new_df.head()

,1 Stars,2 Stars,3 Stars,4 Stars,5 Stars,Brand,Name,Price,Product Details,Product Details Alt,Review Count,Star Rating,Styles,Url
0,1 star (10%),2 star (7%),3 star (9%),4 star (15%),5 star (59%),NaN,Herschel Supply Co. Novel Duffle Bag,65.00,NaN,Product Dimensions: 11 x 20.5 x 11.5 inches; ...,751 customer reviews,4.2 out of 5 stars,Black Grid; Black/Black; Black/Blueprint/White...,NaN
1,1 star (3%),2 star (3%),3 star (4%),4 star (18%),5 star (72%),Ledmark,Ledmark Heavyweight Cotton Canvas Outback Duff...,25.99 - 149.99,NaN,Shipping Information: View shipping rates and...,684 customer reviews,4.6 out of 5 stars,Green; Black,NaN
2,1 star (15%),2 star (7%),3 star (8%),4 star (16%),5 star (54%),NaN,Olympia USA 33 Inch 8 Pocket Rolling Duffel,42.99,NaN,Package Dimensions: 32 x 12 x 5 inches; Shipp...,1158 customer reviews,3.7 out of 5 stars,Black; Black w/ Beige - Exclusive Color; Charc...,NaN
3,1 star (6%),2 star (5%),3 star (8%),4 star (19%),5 star (62%),NaN,"Olympia Luggage 22"" 8 Pocket Rolling Duffel Bag",27.97,NaN,Product Dimensions: 12 x 12 x 22 inches; Ship...,1621 customer reviews,4.3 out of 5 stars,Black; Black / Gray; Black w/ Beige - Exclusiv...,NaN
4,1 star (5%),2 star (6%),3 star (6%),4 star (14%),5 star (69%),Plambag,Plambag Unisex's Canvas Duffel Bag Oversized T...,29.99,NaN,Shipping Weight: 3 pounds (View shipping rate...,739 customer reviews,4.3 out of 5 stars,Grey; Army Green; Coffee; Dark Grey,NaN


In [6]:
def extract_stars(input_string):
    if pd.isnull(input_string):
        return np.nan
    else:
        pattern = re.compile('(\d*)\%')
        percent_string = re.search(pattern, input_string).group(1)
        percent_decimal = int(percent_string)/100
        return percent_decimal
    

In [7]:
new_df['1Star'] = new_df['1 Stars'].apply(extract_stars)
new_df['2Star'] = new_df['2 Stars'].apply(extract_stars)
new_df['3Star'] = new_df['3 Stars'].apply(extract_stars)
new_df['4Star'] = new_df['4 Stars'].apply(extract_stars)
new_df['5Star'] = new_df['5 Stars'].apply(extract_stars)

In [8]:
def extract_rating(input_string):
    if pd.isnull(input_string):
        return np.nan
    else:
        rating = float(input_string[0:3])
        return rating

In [9]:
new_df['Rating'] = new_df['Star Rating'].apply(extract_rating)

In [10]:
def extract_review_count(input_string):
    if pd.isnull(input_string):
        return np.nan
    else:
        pattern = re.compile('(\d*)\s')
        review_count = int(re.search(pattern, input_string).group(1))
        return review_count

In [11]:
new_df['review_count'] = new_df['Review Count'].apply(extract_review_count)

In [12]:
def count_styles(input_string):
    if pd.isnull(input_string):
        return np.nan
    else:
        style_count = len(input_string.split(';'))
        return style_count
        

In [13]:
new_df['style_count'] = new_df['Styles'].apply(count_styles)

In [14]:
def convert_to_dict(input_string):
    if pd.isnull(input_string):
        return np.nan
    else:
        details = {}
        list_details = [ l.split(':') for l in input_string.split(';')]
#         print(list_details)
        for detail_pair in list_details:
            if len(detail_pair) == 2:
                details[detail_pair[0].strip()] = detail_pair[1].strip()
        return details

In [15]:
# Combine the rows where value = NaN
new_df['prod_details'] = new_df[['Product Details', 'Product Details Alt']].applymap(lambda x: '' if pd.isnull(x) else x ).apply(lambda x: x[0] + x[1], axis=1)

In [16]:
new_df['prod_details'] = new_df['prod_details'].map(convert_to_dict)

In [17]:
# Create dictionary to track all product detail categories
d = defaultdict(int)

for detail_dict in new_df['prod_details']:
    if pd.isnull(detail_dict):
        next
    else:
        for ke in detail_dict:
            d[ke] += 1

In [18]:
sorted_keys = sorted(d, key=lambda x: d[x], reverse=True)
for k in sorted_keys[:20]:
    print (k, d[k])

ASIN 12987
Average Customer Review 12252
Amazon Best Sellers Rank 11916
Date first available at Amazon.com 11715
Shipping Weight 11481
Item model number 9643
Product Dimensions 8290
Package Dimensions 2197
Domestic Shipping 1595
International Shipping 1595
Shipping Information 1438
Customer Reviews 735
Best Sellers Rank 674
Item Weight 669
California residents 647
Manufacturer 347
Department 148
Date First Available 94
Shipping Advisory 64
Manufacturer recommended age 55


In [19]:
# new_df['prod_details'][0]

{'ASIN': 'B0077CLHGK',
 'Amazon Best Sellers Rank': '#2442 in Clothing Shoes & Jewelry (See Top 100 in Clothing Shoes & Jewelry)#6 in Clothing Shoes & Jewelry > Luggage & Travel Gear > Travel Duffels',
 'Average Customer Review': '751 customer reviews',
 'Date first available at Amazon.com': 'August 8 2005',
 'Domestic Shipping': 'Item can be shipped within U.S.',
 'International Shipping': 'This item is not eligible for international shipping. Learn More',
 'Item model number': '10026-00007-OS',
 'Product Dimensions': '11 x 20.5 x 11.5 inches',
 'Shipping Advisory': 'This item must be shipped separately from other items in your order. Additional shipping charges will not apply.',
 'Shipping Weight': '2.15 pounds (View shipping rates and policies)'}

In [20]:
def get_asin(details_dict):
    if pd.isnull(details_dict):
        asin = np.nan
    else:
        try:
            asin = details_dict['ASIN']
        except:
            asin = np.nan
#     print(asin)
    return asin

In [21]:
new_df['asin'] = new_df['prod_details'].map(get_asin)

B0077CLHGK
B00BHLBCTO
B075M3XB74
B00144MBWQ
B071CGD1TX
B003KR1K4A
B07543BV2D
B075LGCZNR
B00BT79GOY
B000SRL6TY
B01GGNW2WI
B013M38P8A
B075LGCX7Q
B01AUIAOJ8
B06ZZ4KXT2
B00XDI6ZZU
B072JX711F
B01GWKQO0G
B077STL46R
B0051DV35W
B009ZNI49C
B01GQIUUFY
B008PH8E9Y
B06Y4NNXPX
B019RTW7BE
B01E7W7VP8
B002B4K5Z8
B008HY6HSA
B01GWLHETK
B01M3PQPFF
B00AYXKL9S
B018MV1K8Y
B01724Z47K
B00029R89K
B01CEAHSFS
B07797T3KG
B01HSQ1ITA
B007UQ81CG
B01B7MUDPG
B00ES8KVT0
B01GHPH31U
B019RTVV3Y
B003EZXP5U
B01H5DYKD2
B00LY0HSFA
B01AHSQ6L6
B01J24HCL4
B005HOMZLG
B016IO9GXI
B01NCKSPYG
B06ZYM9W1G
B071LMBW2K
B014TDFMVK
B01M7R9FID
B006VX4KVU
B01N35VVPD
B01GGNW5AM
B00GT5NIF4
B01GWMKRCK
B00TYFSVNU
B074XX2KZW
B01HIGO3OC
B01LYM1APJ
B01FG9WO3K
B004OQ79SW
B01AJSITXW
B003F1L7VC
B071ZZVN7C
B00RZV0I1I
B007AGZ9SK
B072J8WTRV
B0042Z12KQ
B00AYWMQ42
B07528KCYM
B005N26M5C
B0754FQMNK
B002OP5TUA
B01GWKQMZI
B010AOMYXG
B00UIDXFMO
B01LWJU7OF
B0722HML98
B0046CJCH0
B00AYW31WI
B002HG47FY
B00EED8Q96
B011NM2NY4
B01DQ1G288
B075L5KWW6
B000HKMXTO
B00E1O34J0

B01M5BEI1Y
B06XSHR7N1
B01G7IQW50
B00IKW8HIM
B075N94RC3
B01356RXQ4
B001TYI2VK
B01N32USVF
B01D7BTAHC
B074KSJTQN
B01AIP876K
B004HSWL62
B00MNQZ39C
B001E0BMNY
B077Z2P9VH
B00T95J580
B01BNZX624
B00BUZN0LA
B01HAZB4PW
B071VGNNFJ
B01N2IF1OM
B06XYW43B3
B01JGL7SKS
B078S49PT8
B078XRYVH8
B00VWKS6DU
B01HFQTBZ6
B073ZGC7B4
B071NGSM7M
B0744BJSYD
B00E67C05Q
B00M4GFPGW
B075JC4HY8
B00S1F5DZI
B0754QCDHM
B017BNE9TG
B00IMXRYRY
B00CJIFYS4
B076YC2MRL
B000E94SA0
B00EART6XQ
B00XQB9PRE
B00HYJP6PY
B017I50R66
B01N56WRT7
B00N8IOOC6
B074W5V3KZ
B074HK4MS2
B075KWW325
B01MXIP8PM
B01IDNZOGK
B00VRETJD2
B01COKDINY
B00PSMXE62
B0797YQRJ8
B0000YXWA6
B001OL3QGY
B01MTYCBHK
B00VF3LQMW
B076Y7M49P
B01AB252OA
B004NXBGOE
B0011B9FTO
B00A928CF4
B075FXXR2Q
B01N5MKQRO
B01N6Z83TP
B01KPMED78
B01FKUZHFC
B016MUI710
B00IMO05A6
B01BTGX50K
B0063XOEFK
B01MAV2MDW
B004AGAV5Y
B00NL0DQP2
B01G4RKFGG
B01L310D6K
B077VR39B2
B01D57N3P8
B01JGL7V6O
B06XZF9PRM
B00SYFMWJ0
B07578C55T
B01E8OHB50
B01GVITBSG
B00JG66N64
B073SP4DT2
B0037UWGEI
B07143QLFQ
B075VFGHWD

B00P1RZC8W
B016QBS0IU
B076H8ZR4T
B018O5GEYS
B06Y1Q56J7
B00A3LXKU8
B074J8XBJ2
B071K8Y5KT
B074H12LVX
B074SMLYQR
B005GUQ5Y4
B019JHMSCM
B01MFH4IIN
B01171PI7U
B0046XGZ8S
B00EBYVKLY
B00VF3HJD2
B073RH2TR1
B078JBG91D
B00FC48YE8
B01HU15EBQ
B077KHVQ1S
B074JGY5ZS
B00OMA2NRW
B00IVSH8JO
B00R0LHRW6
B00SYFMWKE
B007K4FMKC
B01MU371OQ
B01IE2TJMK
B01M116Q4J
B00HY6C6YQ
B01JTB9TJS
B006MKHF5U
B01N2YP173
B06XPY8X7W
B073TNQZQT
B06XPY8X7W
B073TNQZQT
B00YF1S71E
B016KWMOQO
B000E9BR3G
B075SMQ72X
B0017V94J4
B00NITDJAS
B074JGGNMF
B00IJC0PWO
B06VWRMJWK
B00O7ZAOMI
B00EZ8648K
B073NWB32L
B00Q2DRF34
B01K08BBSG
B004ZHJ4TM
B01EYE7FRS
B00FPRSB8G
B01MUY4FFA
B016MUHXV0
B00U0N43Y6
B016MUI33C
B018V8AIOU
B072JXHKKR
B01M116Q4W
B008BUDEUY
B00BXOR578
B0772QFHVR
B014Z54RT0
B06XHSQWMK
B0069ZTVQ4
B003MNSO28
B01LX4HZMO
B0132EYTSO
B000IBQOHE
B00VETM162
B0778HYBN6
B01J4JO9OA
B017IR7CTY
B074KSB1HC
B01N3AQWZD
B018O5GD30
B01E7S9W5E
B01M1E3M8A
B000CEFSRY
B01E7S8C8W
B0057KW9K2
B01FIVQEOQ
B00Y00FGDW
B01E7S7HLA
B071S6DLDC
B06Y1N3KTS
B07771L335

B00KGKQ3ZU
B07117ZS6D
B00KAIWZDM
B00L6DYL8M
B0755YRS3T
B073WVC786
B01MSDXNUE
B00I80EK14
B01GPCPLM8
B00MEZKF1I
B01M7PTSIQ
B014R9GUXA
B0765F3SS9
B016QO688G
B01N65UFXL
B0757CXKX2
B01LXOM72M
B078Y3579R
B07427VMPX
B00KJ6EFLU
B01FW7UT9C
B06X965R8G
B00NHK0CTE
B01I8GX1H6
B0757CWQGK
B01LW5R53D
B0771H8D4F
B00MX45VJG
B00UJXXND4
B07816N8SX
B073P48X3G
B06XYG1BT5
B06XC3JK2V
B075FCMZ9D
B00380X3A8
B015GJ256G
B0739WDTRV
B07426PH7D
B075WRRLMH
B074G3NMS6
B072JV8X1S
B071VR2V7Y
B002IAH7MY
B01EIOICW6
B06ZXZ9F6K
B01DUNKKJ4
B07255LGP3
B0765D7YKF
B07369W56J
B00LGWG320
B000I01DHQ
B00I80ELKO
B00T5FEG1U
B074XT42NH
B074T8HYLJ
B06XQDMKQ6
B003VVDZ5M
B00V3CCKZM
B071JB7918
B010YDVG1Y
B071JNHYYW
B01GJEK18G
B01KVCNCYW
B075K529ZB
B00T5FEG1U
B075ZHCXGV
B00SJO46YA
B019PEN1XY
B01N268P0O
B00OV4IQP2
B076GR4JZ5
B00H346S7A
B01KKCD52C
B078SKQN8R
B0772KXZ4Q
B012CKXVWO
B00J8TRKH0
B01JSBKP6K
B06XD97V11
B01MRLRTZW
B06XJ1HMG5
B0757Q668W
B076BNQNSR
B01J81PXCG
B00VX1K5OG
B071FVM82V
B073X5GTP7
B006S74GUY
B015GIMDNC
B01GP9SWK4
B074ZSWQNB

B00PCQIFWW
B014VL66V0
B01N9UWE2F
B01N9UWE2F
B0722K1FPZ
B01BWXT1G2
B00Y3Q79WY
B076HFNPKQ
B073GB4BXR
B0722W4JKP
B0082CA1SO
B00JANYOFK
B00GDNIE7Y
B00ICSIM02
B0733LVTSL
B00CHRY0N2
B0027A7HUW
B07175QD9Q
B01L2KICCO
B0095VEPME
B01F53FBXC
B074S4JBKW
B00KGWTVCA
B01M0428NH
B075VPZ9PS
B00QMVA5CE
B00ES0NF4G
B074RY5FXC
B00WM8ZF8U
B017TL993G
B01L9CNT96
B01AJPRA1M
B00PBDWNCO
B00LNQ0RQW
B0786ZLCKR
B00GDA1QWC
B000BQM862
B01LMJMFPA
B0160A8ZZK
B004D18JLY
B01N223X4R
B00CJKUEIM
B00V98514M
B00O3FZZ44
B00SEW3G7A
B01FMH74TK
B075FZG9YG
B01MU9D2DU
B01LYFF7PP
B071ZV4R15
B00VENCE6K
B071ZV4R15
B00VENCE6K
B016EAGK1W
B014Q38JYK
B00MHYLI1W
B005N4H1I2
B06XV9B6F9
B01M4FQV40
B01N5QDQJI
B00VGWHMU2
B071KT8FSM
B0716P8BBN
B077T1RPT7
B012H8UM78
B01BO6580A
B00SQ34KFY
B076JRSTL8
B072FNKR59
B00YRK3G7S
B0743JXPQ9
B006QOJ4MY
B078HQ67ZL
B00DO1XFSQ
B00S1WE0W8
B001IY2G04
B06XTJBLVC
B01ETTPNMW
B00YEACR1W
B00V8PHEY6
B071HC39Y8
B00YHNO96C
B01N4APRI3
B00CVGC54K
B007SANM96
B073B33VMQ
B01A7IFWMK
B001VZWQQY
B07835SHBW
B071DGMJBB
B072LTVX51

B06Y1T7GMV
B074HCSRR1
B076VPRXCW
B00WKJ6JTU
B06Y2LL2MJ
B00BMFHTGA
B00L7RV4I2
B01CI27FGO
B01BIL2O8K
B018HLNK7I
B075SGLSR8
B0097HBD80
B06XB3Q2DZ
B01AXMSG6E
B01F84Y0R6
B06XWTQ6TF
B01JWGMHR6
B001GDUBA4
B00MW7W56Q
B01NBQCJ6K
B01AHSPSZ6
B00IJGLJAW
B075DKF67F
B075WXX9V9
B008MIPKNE
B01EJTTINW
B01N78EB9D
B008CCSRXK
B014R9FTB4
B00114RFCU
B00G0NTS9K
B01LXSUOXD
B00HP8ZMWQ
B01LXV36MZ
B074FFNT8J
B017BNKWQ0
B00A6JCOEU
B01C34LG4E
B01JGOALV8
B071HMW9ZS
B01MYOOOCM
B06XY8M7L6
B07688TF38
B07791YHY4
B01G87I63G
B0745ZBSSQ
B00JS0TTMI
B00NDAUCXY
B073YX3TVG
B00Z28BZTK
B01M5DGS68
B01M5DGS68
B00NP9IJVA
B012FXT0D2
B006MXK3JM
B003EST4MK
B00KKV8QNW
B071Z8835K
B005GUSRWC
B074MCVCP5
B071WTSPT7
B06X9XZG9X
B01M1F9ILA
B01LVZC2CO
B06W2LZK4W
B00L1H4AN4
B077QXSNRZ
B00XFIFANQ
B01DYFYVKM
B017ANM4VC
B0040TQRM2
B06XPCXH4N
B01M5LFQKG
B003Y851JU
B001GDR1XY
B01ABDDPGQ
B019XER6A0
B074JJY94F
B00VGWHJ1O
B07588MPBV
B00CAAC4GQ
B01LX0C5U8
B01FQVJJ5E
B00I81CKOW
B00PT5Q7L2
B01MA5XTD0
B06XG2RTH7
B06X9PHDS7
B00JQQGYUE
B00QYUBWFM
B077NWLDZN

B01LYY784I
B0744JSY64
B01N986I5H
B01LY8CUSP
B06XPQ1D25
B016IXXIHO
B00X6JS2LG
B00W9BTOFU
B01BH8HYIO
B01FV5824O
B0753ZT5TM
B077W5JBLC
B0029M0UFM
B01N96CLHL
B013TA1ARE
B0751KQMZ3
B01L0E92X0
B077H2G722
B00PVMBC7W
B078VL3LZD
B0742GCJTV
B0037IUX86
B01H5PWIES
B01MR48SCX
B00YFBCQCA
B00TKT4WZG
B01N9CCGW0
B01L0IX50C
B073VYB4RY
B00YYK3UCM
B00BKUV2L0
B06Y1B3R4X
B01CXGFKYO
B005WEH0O8
B01MYDQ7MC
B01LYK8ZRF
B01I3H8VO8
B0037WUEKO
B002LV8F30
B001Q6RPKU
B0711CLKJZ
B073WGXS9Z
B071DVRDSG
B00SWR4UJU
B00ISI6UGY
B01M7NPSJK
B0773VZS5Z
B073V3CK4K
B06XGVX58X
B01LZXG8JR
B01EZEPUN8
B078T4T8BP
B01MR48OM7
B078KDXHR5
B00C2F25JA
B06XYQK4MX
B0765L8HFY
B07117FY2D
B01126SZO8
B074Y7498H
B01BUCGXAW
B06XH2SKF7
B01KMTNJVK
B0050JUEFM
B0763VDT4X
B071NL7853
B003GXG31S
B077M8WKVM
B008M6ZDY2
B000NQ5BN2
B01N1N2BUR
B00K1W5368
B01H5PWEWY
B00CDWPYI6
B00SWR4UB8
B077H262Q6
B0148NDJ2K
B0001ZI4KW
B076TQ75GK
B072Q17K4H
B0744GQGFN
B00N2XIUK4
B00S7EB5Q4
B01H41QUVU
B074NSTMTZ
B0771F61LS
nan
B075FLBWSF
B01N5MFE7Y
B075FD9SKN
B074T8RWTJ
B00Y11

B002W89IBU
B01AOEB1X6
B00WBFGBFU
B0746HXWYH
B0718Z38JK
B072Q8BV42
B00S6YNLSU
B00ICD5SPY
B00DOMM1LW
B01LBXB6CA
B01M3SRNDF
B00L1DL360
B06ZY7KB9V
B00NXL554Y
B01HFH461Y
B01N22S5C4
B076TH2VQN
B01MXPQIBA
B077SX7Y5T
B00T5D0E86
B075WYTMRJ
B073WH6J9L
B06XTKKRSG
B010Y1LIJG
B0721HVT2B
B01N96K8EN
B071SH27KD
B01MTQGKR1
B01DJNWQNS
B01ATR68W2
B06XCZ9T79
B076MJWTXG
B06XDMGDGN
B01LY9KO7N
B075KGMC1Q
B00YRBNY1K
B016ZVGBGY
B01N80IKRG
B075ZVVB6V
B074F4JD38
B074ZRPSC7
B01LR61OQE
B072M4NT1V
B071JM7FD2
B00GFZL710
B07556VZXM
B01DUYSIE2
B00AE0NM5G
B074PWR515
B0765Q7BJ3
B0711GXYVD
B072Q5BZ8G
B076ZGNTMS
B01MRAJM1N
B01LFT3F4M
B073DD3ZYZ
B01I01YX76
B01M9A8YI9
B00QHCNQDI
B07211SSK9
B015WKETLI
B0081VWEKO
B00LBI6ZMW
B00AKTKTSO
B01E5JKNMG
B01MA6N1D2
B01CG1OP1K
B00ARM2ZNG
B00KO61PS6
B07173T8LJ
B00YXTXGW8
B06Y4BZ6CV
B073J2HQVJ
B071ZWPD1W
B06Y2TSS5Z
B077XBSSHR
B00SUI3L8M
B06WGPN9S5
B012BF39AE
B076CFDF3L
B071S52M6F
nan
B076MHFHMB
B01MUGMFSZ
B0761Z9F1B
B00NG4VWTA
B0735G772W
B00MOF664S
B01HGQHQSE
B01J5CECTI
B00PF5LR3E
B00LNI

B015SWEPBE
B01CKMABPO
B07693XJZC
B014ALRXC2
B01DTL9NEK
B071YLFMHT
B00WJ6TN3I
B074DQ833B
B00ENJ810K
B00S8MVSQW
B074T2L5B8
B01N8TUUSE
B01E4OON1Y
B01MTVS552
B00CIC00EY
B002JGMKFQ
B00W5TRXSQ
B01MTORTDV
B075K61X8G
B00BCRUUSC
B00JU3AVFC
B01352VI0K
B073C2RTP8
B00OTR5KGY
B00QAZX5CY
B072PXCWVG
B00MOF8N9O
B06Y62DMM3
B072X8B9XM
B012891BYE
B06WWQY4G9
B00DOSUCRQ
B007IFOADI
B001TYBYYC
B07575BTKT
B000B836CA
B003GCXSO4
B0778W21RZ
B016RGLGLM
B00TTNG6HA
B078PFH2JL
B00EJH4U2O
B01GWMH0SY
B00821M1OM
B01M16QAI8
B00HJEE6VO
B00EYXS984
B01LYR3BKQ
B07434ZJXP
B00PFG23K4
B01KMTAFK8
B0772PKWN6
B076HC28LY
B000CC4VJ2
B0771LZNV6
B01FKCQKF6
B07845ZGSJ
B01N3A79PF
B071Y3Y3TF
B073QM8PV9
B075WRQRKV
B077KDWBFW
B0731PW57Y
B01MTY6AU0
B01GGJL5H0
B06Y183H3Y
B073RRJWWX
B071S3NB2Y
B01F26KHF4
B013WOG2MA
B077H51QXK
B071FNF91R
B077SSD836
B06XBHVXHG
B00FOLZV90
B01N9THTVG
B073DG983C
B071LDNQ4J
B01N79IQLN
B0744FQTN5
B00UZHYVU8
B077J61VJF
B002NW7KKQ
B00AAY0DXU
B01EJ42TPG
B01E6VZQQ6
B074XXT269
B075DG56H2
B001A74OBI
B06XN5MLFW
B0079QHJH0

In [22]:
def get_rank(details_dict):
    if pd.isnull(details_dict):
        ranking = np.nan
    else:
        try:
            ranking = details_dict['Amazon Best Sellers Rank']
        except:
            try:
                ranking = details_dict['Best Sellers Rank']
            except:
                ranking = np.nan
#     print(ranking)
    return ranking

new_df['amazon_ranking'] = new_df['prod_details'].map(get_rank)

In [23]:
# get date launch
def get_date(details_dict):
    if pd.isnull(details_dict):
        date = np.nan
    else:
        try:
            date = details_dict['Date first available at Amazon.com']
            date = pd.to_datetime(date)
        except:
            date = np.nan
    return date

new_df['date_launch'] = new_df['prod_details'].map(get_date)
new_df['days_since_launch'] = (pd.to_datetime('today') - new_df['date_launch']).map(lambda x: x.days)

In [24]:
# Convert styles to list
new_df['styles'] = new_df['Styles'].str.split(';')

In [25]:
# Extract weight
def get_weight(details_dict):
    if pd.isnull(details_dict):
        weight = np.nan
    else:
        try:
            weight_string = details_dict['Shipping Weight'].strip()
            weight_list = weight_string.split('(')[0].split(' ')
            if weight_list[1] == 'pounds':
                weight = float(weight_list[0]) * 16
            elif weight_list[1] == 'ounces':
                weight = float(weight_list[0])
            else:
                weight = np.nan
                print(weight_list)
        except:
            weight = np.nan
#         print(weight)
    return weight



In [26]:
new_df['weight_ounces'] = new_df['prod_details'].map(get_weight)

In [27]:
def get_volume(details_dict):
    volume_unit = np.nan
    volume_string = np.nan
    if pd.isnull(details_dict):
        volume = np.nan
    else:
        try:
            if details_dict['Product Dimensions']:
                volume_string = details_dict['Product Dimensions']
#             elif details_dict['Package Dimensions']:
#                 volume_string = details_dict['Package Dimensions']
            volume_list = volume_string.split('x')
            last_unit = volume_list[-1]
            [volume_list[-1], volume_unit] = last_unit.strip().split(' ')
            volume_list = [float(num.strip()) for num in volume_list]
            volume = np.prod(volume_list)
        except:
            volume = np.nan
    return volume_string, volume, volume_unit, 

In [28]:
volume_results = pd.DataFrame(list(new_df['prod_details'].map(get_volume)))

In [29]:
new_df['volume_dimensions'], new_df['volume'], new_df['volume_unit']= volume_results[0], volume_results[1], volume_results[2]

In [30]:
# Verify that all volumes are in inches
new_df[(new_df['volume_unit'].notnull()) & (new_df['volume_unit'] != 'inches')]

,1 Stars,2 Stars,3 Stars,4 Stars,5 Stars,Brand,Name,Price,Product Details,Product Details Alt,...,prod_details,asin,amazon_ranking,date_launch,days_since_launch,styles,weight_ounces,volume_dimensions,volume,volume_unit


In [31]:
def get_min_max_price(input_str):
    if pd.isnull(input_str):
        min_price, max_price = np.nan, np.nan
    else:
        min_price, max_price = np.nan, np.nan
        prices = input_str.split('-')
        if len(prices) >= 1:
            min_price = float(prices[0])
        if len(prices) >= 2:
            max_price = float(prices[1])
    return min_price, max_price

In [32]:
price_results = pd.DataFrame(list(new_df['Price'].map(get_min_max_price)))

In [33]:
new_df['min_price'], new_df['max_price'] = price_results[0], price_results[1]

In [34]:
def get_first_ranking(string):
    if pd.isnull(string):
        rank = np.nan
        category = np.nan
    else:
        try:
            first_ranking = string.split('(')[0].split(' ')
            rank = first_ranking[0].replace('#', '')
            category = (' ').join(first_ranking[2:])
#             print(rank, category)
        except:
            rank = np.nan
            category = np.nan
    return rank, category

In [35]:
rankings_results = pd.DataFrame(list(new_df['amazon_ranking'].map(get_first_ranking)))

2442 Clothing Shoes & Jewelry 
1807 Sports & Outdoors 
2324 Clothing Shoes & Jewelry 
1131 Clothing Shoes & Jewelry 
1169 Sports & Outdoors 
2087 Clothing Shoes & Jewelry 
6486 Sports & Outdoors 
931 Sports & Outdoors 
7933 Clothing Shoes & Jewelry 
1962 Clothing Shoes & Jewelry 
2141 Clothing Shoes & Jewelry 
1621 Sports & Outdoors 
2599 Sports & Outdoors 
6767 Clothing Shoes & Jewelry 
11019 Clothing Shoes & Jewelry 
98224 Clothing Shoes & Jewelry 
17117 Clothing Shoes & Jewelry 
9681 Clothing Shoes & Jewelry 
4516 Clothing Shoes & Jewelry 
18531 Sports & Outdoors 
9005 Clothing Shoes & Jewelry 
133826 Clothing Shoes & Jewelry 
14728 Clothing Shoes & Jewelry 
7083 Clothing Shoes & Jewelry 
5493 Sports & Outdoors 
9408 Clothing Shoes & Jewelry 
12794 Clothing Shoes & Jewelry 
19190 Clothing Shoes & Jewelry 
3476 Sports & Outdoors 
41214 Clothing Shoes & Jewelry 
4514 Sports & Outdoors 
7245 Clothing Shoes & Jewelry 
29707 Clothing Shoes & Jewelry 
6626 Clothing Shoes & Jewelry 
15566 

131606 Sports & Outdoors 
34376 Sports & Outdoors 
108859 Clothing Shoes & Jewelry 
517018 Clothing Shoes & Jewelry 
315729 Clothing Shoes & Jewelry 
562838 Clothing Shoes & Jewelry 
140439 Clothing Shoes & Jewelry 
646612 Clothing Shoes & Jewelry 
415 Clothing Shoes & Jewelry > Luggage & Travel Gear > Travel Duffels
156450 Sports & Outdoors 
736534 Clothing Shoes & Jewelry 
384716 Sports & Outdoors 
578160 Clothing Shoes & Jewelry 
370493 Clothing Shoes & Jewelry 
683828 Clothing Shoes & Jewelry 
143536 Arts Crafts & Sewing 
597138 Clothing Shoes & Jewelry 
457545 Clothing Shoes & Jewelry 
107397 Sports & Outdoors 
493422 Clothing Shoes & Jewelry 
84694 Sports & Outdoors 
415362 Clothing Shoes & Jewelry 
1041323 Clothing Shoes & Jewelry 
577046 Clothing Shoes & Jewelry 
379736 Clothing Shoes & Jewelry 
229789 Clothing Shoes & Jewelry 
321015 Clothing Shoes & Jewelry 
694817 Clothing Shoes & Jewelry 
966186 Clothing Shoes & Jewelry 
886 Clothing Shoes & Jewelry > Luggage & Travel Gear 

515498 Sports & Outdoors 
1051866 Clothing Shoes & Jewelry 
1201928 Clothing Shoes & Jewelry 
1733445 Clothing Shoes & Jewelry 
961515 Clothing Shoes & Jewelry 
745992 Clothing Shoes & Jewelry 
1955044 Clothing Shoes & Jewelry 
2699606 Clothing Shoes & Jewelry 
1088689 Clothing Shoes & Jewelry 
816650 Clothing Shoes & Jewelry 
241208 Clothing Shoes & Jewelry 
1051862 Clothing Shoes & Jewelry 
420260 Sports & Outdoors 
1581299 Clothing Shoes & Jewelry 
1097474 Clothing Shoes & Jewelry 
3101786 Clothing Shoes & Jewelry 
958084 Clothing Shoes & Jewelry 
199330 Clothing Shoes & Jewelry 
904645 Clothing Shoes & Jewelry 
811256 Clothing Shoes & Jewelry 
1575946 Clothing Shoes & Jewelry 
962401 Clothing Shoes & Jewelry 
999936 Clothing Shoes & Jewelry 
1197570 Clothing Shoes & Jewelry 
424135 Clothing Shoes & Jewelry 
754061 Clothing Shoes & Jewelry 
288162 Clothing Shoes & Jewelry 
279374 Clothing Shoes & Jewelry 
1512413 Home & Kitchen 
896385 Clothing Shoes & Jewelry 
930693 Clothing Shoes

1243140 Clothing Shoes & Jewelry 
1062331 Clothing Shoes & Jewelry 
837296 Sports & Outdoors 
1465522 Clothing Shoes & Jewelry 
442351 Clothing Shoes & Jewelry 
1034929 Clothing Shoes & Jewelry 
1460327 Clothing Shoes & Jewelry 
2475996 Clothing Shoes & Jewelry 
402414 Sports & Outdoors 
1653998 Clothing Shoes & Jewelry 
1674250 Clothing Shoes & Jewelry 
1749316 Clothing Shoes & Jewelry 
646312 Sports & Outdoors 
2051124 Clothing Shoes & Jewelry 
834308 Sports & Outdoors 
2003663 Clothing Shoes & Jewelry 
1584550 Clothing Shoes & Jewelry 
3548487 Clothing Shoes & Jewelry 
223650 Sports & Outdoors 
2382265 Clothing Shoes & Jewelry 
624134 Clothing Shoes & Jewelry 
819818 Clothing Shoes & Jewelry 
3186609 Clothing Shoes & Jewelry 
1668412 Clothing Shoes & Jewelry 
1829678 Clothing Shoes & Jewelry 
1292499 Clothing Shoes & Jewelry 
2286622 Clothing Shoes & Jewelry 
2400829 Clothing Shoes & Jewelry 
1388623 Clothing Shoes & Jewelry 
2337626 Clothing Shoes & Jewelry 
1426167 Clothing Shoes 

944727 Sports & Outdoors 
896746 Clothing Shoes & Jewelry 
2964212 Clothing Shoes & Jewelry 
534954 Sports & Outdoors 
4305497 Clothing Shoes & Jewelry 
15426056 Clothing Shoes & Jewelry 
4276020 Clothing Shoes & Jewelry 
1625834 Clothing Shoes & Jewelry 
2953902 Clothing Shoes & Jewelry 
3521996 Clothing Shoes & Jewelry 
3338058 Clothing Shoes & Jewelry 
629564 Clothing Shoes & Jewelry 
849014 Sports & Outdoors 
4027738 Clothing Shoes & Jewelry 
4846137 Clothing Shoes & Jewelry 
1335122 Sports & Outdoors 
2371956 Clothing Shoes & Jewelry 
5616968 Clothing Shoes & Jewelry 
4947182 Clothing Shoes & Jewelry 
3615602 Clothing Shoes & Jewelry 
2185924 Clothing Shoes & Jewelry 
718094 Sports & Outdoors 
347385 Electronics 
3891664 Clothing Shoes & Jewelry 
972838 Sports & Outdoors 
481334 Sports & Outdoors 
3260257 Clothing Shoes & Jewelry 
958948 Sports & Outdoors 
4935676 Clothing Shoes & Jewelry 
4104606 Clothing Shoes & Jewelry 
3477789 Clothing Shoes & Jewelry 
2562689 Clothing Shoes &

...

864821 Clothing Shoes & Jewelry 
756443 Clothing Shoes & Jewelry 
328896 Clothing Shoes & Jewelry 
311182 Clothing Shoes & Jewelry 
127448 Clothing Shoes & Jewelry 
307810 Clothing Shoes & Jewelry 
188412 Clothing Shoes & Jewelry 
557084 Clothing Shoes & Jewelry 
495638 Clothing Shoes & Jewelry 
406645 Clothing Shoes & Jewelry 
650427 Clothing Shoes & Jewelry 
361684 Clothing Shoes & Jewelry 
432199 Clothing Shoes & Jewelry 
470484 Clothing Shoes & Jewelry 
183814 Clothing Shoes & Jewelry 
895748 Clothing Shoes & Jewelry 
415923 Clothing Shoes & Jewelry 
199760 Clothing Shoes & Jewelry 
99996 Clothing Shoes & Jewelry 
1159583 Clothing Shoes & Jewelry 
481201 Clothing Shoes & Jewelry 
568179 Clothing Shoes & Jewelry 
567718 Clothing Shoes & Jewelry 
397530 Clothing Shoes & Jewelry 
868998 Clothing Shoes & Jewelry 
1148838 Home & Kitchen 
302352 Clothing Shoes & Jewelry 
145 Clothing Shoes & Jewelry > Luggage & Travel Gear > Luggage > Luggage Sets
744061 Clothing Shoes & Jewelry 
674822 

715641 Clothing Shoes & Jewelry 
1293509 Clothing Shoes & Jewelry 
107323 Clothing Shoes & Jewelry 
444013 Clothing Shoes & Jewelry 
580036 Clothing Shoes & Jewelry 
874151 Clothing Shoes & Jewelry 
715082 Clothing Shoes & Jewelry 
1110724 Clothing Shoes & Jewelry 
803212 Clothing Shoes & Jewelry 
688885 Clothing Shoes & Jewelry 
1192730 Clothing Shoes & Jewelry 
310125 Clothing Shoes & Jewelry 
1270048 Clothing Shoes & Jewelry 
760554 Clothing Shoes & Jewelry 
601549 Clothing Shoes & Jewelry 
470740 Clothing Shoes & Jewelry 
142844 Sports & Outdoors 
1101395 Clothing Shoes & Jewelry 
1039517 Clothing Shoes & Jewelry 
964216 Clothing Shoes & Jewelry 
871247 Clothing Shoes & Jewelry 
417182 Clothing Shoes & Jewelry 
200295 Health & Household 
251133 Clothing Shoes & Jewelry 
1065587 Clothing Shoes & Jewelry 
746921 Clothing Shoes & Jewelry 
609319 Clothing Shoes & Jewelry 
58992 Clothing Shoes & Jewelry 
1130991 Clothing Shoes & Jewelry 
567135 Clothing Shoes & Jewelry 
911585 Clothing 

1468341 Clothing Shoes & Jewelry 
185136 Clothing Shoes & Jewelry 
262924 Clothing Shoes & Jewelry 
907541 Clothing Shoes & Jewelry 
1122435 Clothing Shoes & Jewelry 
748500 Clothing Shoes & Jewelry 
228999 Clothing Shoes & Jewelry 
1122258 Clothing Shoes & Jewelry 
777 Clothing Shoes & Jewelry > Luggage & Travel Gear > Luggage > Carry-Ons
126200 Patio Lawn & Garden 
243230 Clothing Shoes & Jewelry 
1030382 Clothing Shoes & Jewelry 
605451 Clothing Shoes & Jewelry 
296280 Clothing Shoes & Jewelry 
534367 Clothing Shoes & Jewelry 
1303256 Clothing Shoes & Jewelry 
966401 Clothing Shoes & Jewelry 
356385 Clothing Shoes & Jewelry 
629705 Clothing Shoes & Jewelry 
1482676 Clothing Shoes & Jewelry 
441341 Clothing Shoes & Jewelry 
205545 Electronics 
1002165 Clothing Shoes & Jewelry 
2255223 Clothing Shoes & Jewelry 
1129263 Clothing Shoes & Jewelry 
599778 Clothing Shoes & Jewelry 
934553 Clothing Shoes & Jewelry 
1249069 Clothing Shoes & Jewelry 
177336 Clothing Shoes & Jewelry 
949673 Cl

1059887 Clothing Shoes & Jewelry 
960235 Home & Kitchen 
364684 Clothing Shoes & Jewelry 
328293 Clothing Shoes & Jewelry 
945770 Clothing Shoes & Jewelry 
2109581 Clothing Shoes & Jewelry 
478427 Clothing Shoes & Jewelry 
1942902 Clothing Shoes & Jewelry 
1213412 Clothing Shoes & Jewelry 
2212939 Clothing Shoes & Jewelry 
1266304 Clothing Shoes & Jewelry 
1437818 Clothing Shoes & Jewelry 
191677 Clothing Shoes & Jewelry 
1322330 Clothing Shoes & Jewelry 
259570 Clothing Shoes & Jewelry 
1511908 Home & Kitchen 
1492130 Clothing Shoes & Jewelry 
1783427 Clothing Shoes & Jewelry 
2433732 Clothing Shoes & Jewelry 
1469987 Clothing Shoes & Jewelry 
946677 Clothing Shoes & Jewelry 
210133 Clothing Shoes & Jewelry 
32880 Electronics 
1281984 Clothing Shoes & Jewelry 
284362 Clothing Shoes & Jewelry 
615939 Clothing Shoes & Jewelry 
1743394 Clothing Shoes & Jewelry 
1530103 Clothing Shoes & Jewelry 
338813 Sports & Outdoors 
913827 Clothing Shoes & Jewelry 
1077391 Clothing Shoes & Jewelry 
1

566047 Clothing Shoes & Jewelry 
2249469 Clothing Shoes & Jewelry 
729192 Clothing Shoes & Jewelry 
1482974 Clothing Shoes & Jewelry 
393413 Clothing Shoes & Jewelry 
1526006 Clothing Shoes & Jewelry 
2296237 Clothing Shoes & Jewelry 
1660396 Clothing Shoes & Jewelry 
1487265 Clothing Shoes & Jewelry 
1598205 Clothing Shoes & Jewelry 
1299436 Clothing Shoes & Jewelry 
1510656 Clothing Shoes & Jewelry 
1852665 Clothing Shoes & Jewelry 
531177 Clothing Shoes & Jewelry 
669607 Clothing Shoes & Jewelry 
385652 Clothing Shoes & Jewelry 
472925 Sports & Outdoors 
2186838 Clothing Shoes & Jewelry 
1654968 Clothing Shoes & Jewelry 
646 Clothing Shoes & Jewelry > Luggage & Travel Gear > Travel Totes#2433 in Clothing Shoes & Jewelry > Luggage & Travel Gear > Luggage
2138508 Sports & Outdoors 
574939 Sports & Outdoors 
15371301 Clothing Shoes & Jewelry 
574000 Clothing Shoes & Jewelry 
1968030 Clothing Shoes & Jewelry 
1012541 Clothing Shoes & Jewelry 
617208 Sports & Outdoors 
1756543 Clothing S

In [36]:
new_df['amazon_rank'], new_df['rank_category'] = rankings_results[0], rankings_results[1]

In [37]:
new_df.drop(['Price', '1 Stars', '2 Stars', '3 Stars', '4 Stars', '5 Stars', 'Product Details', 'Product Details Alt', 'Star Rating', 'Review Count', 'Styles', 'date_launch'], axis=1, inplace=True)

In [39]:
new_df.columns

Index(['Brand', 'Name', 'Url', '1Star', '2Star', '3Star', '4Star', '5Star',
       'Rating', 'review_count', 'style_count', 'prod_details', 'asin',
       'amazon_ranking', 'days_since_launch', 'styles', 'weight_ounces',
       'volume_dimensions', 'volume', 'volume_unit', 'min_price', 'max_price',
       'amazon_rank', 'rank_category'],
      dtype='object')